# Astro II Lab 2 — Cosmic Fate Explorer (Friedmann Dynamics)

**Goal:** use the Friedmann equation to explore how the **destiny** of the universe changes with
$\Omega_r$, $\Omega_m$, $\Omega_\Lambda$ and curvature.

We integrate the **acceleration equation** (Raychaudhuri form):
$$
\ddot{a} = \frac{H_0^2}{2}\!\left(-2\,\Omega_r\, a^{-3}
\;-\; \Omega_m\, a^{-2} \;+\; 2\,\Omega_\Lambda\, a\right),
$$
derived from the full Friedmann equation
$$
H(a)^2 = H_0^2\!\left(\Omega_r\, a^{-4}
+ \Omega_m\, a^{-3} + \Omega_k\, a^{-2} + \Omega_\Lambda\right),
$$
with $\Omega_k = 1-\Omega_r-\Omega_m-\Omega_\Lambda$.

Using the second-order form avoids the $\pm\sqrt{\phantom{x}}$ sign ambiguity and
**naturally handles turnaround + collapse** (Big Crunch).

---

## What to do
1. Set $\Omega_m$, $\Omega_r$, $\Omega_\Lambda$ and $H_0$ with the sliders.
2. **Drag the time slider** to scrub through cosmic history and watch the globe expand or shrink.
3. Identify regimes:
   - **recollapse / Big Crunch**
   - **accelerate forever / heat death**
   - **marginal / coasting**
4. Read off the **age** (time since $a\to 0$) and fate from the info box.

> Key takeaway: **geometry is not the same as fate** — once $\Omega_\Lambda$ is allowed.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display, clear_output

# ═══════════════════════════════════════════════════════════
# Constants
# ═══════════════════════════════════════════════════════════
MPC_IN_KM  = 3.0856775814913673e19   # km per Mpc
SEC_IN_GYR = 3.15576e16              # seconds per Gyr

def H0_invGyr(H0):
    """Convert H0 [km/s/Mpc] to [1/Gyr]."""
    return H0 / MPC_IN_KM * SEC_IN_GYR

# ═══════════════════════════════════════════════════════════
# Friedmann ODE — second-order (Raychaudhuri equation)
#
#   a_ddot = (H0^2/2)(-2 Or a^-3 - Om a^-2 + 2 OL a)
#
# This avoids the +/- sqrt sign ambiguity of the first-order
# form and naturally handles turnaround + collapse.
# ═══════════════════════════════════════════════════════════
def _friedmann_rhs(t, y, H0g, Or, Om, Ol):
    """RHS for the system  y = [a, v=da/dt]."""
    a, v = y
    if a < 1e-12:
        return [0.0, 0.0]
    a_ddot = 0.5 * H0g**2 * (-2.0*Or/a**3 - Om/a**2 + 2.0*Ol*a)
    return [v, a_ddot]


def integrate(Or, Om, Ol, H0=70.0, tmax=200.0):
    """Integrate a(t) backward (Big Bang) and forward (future).

    Returns (t_all, a_all, Omega_k, age_Gyr, fate_string).
    """
    Ok  = 1.0 - Om - Ol - Or
    H0g = H0_invGyr(H0)
    AMIN, AMAX = 1e-4, 50.0

    rhs = lambda t, y: _friedmann_rhs(t, y, H0g, Or, Om, Ol)
    y0  = [1.0, H0g]          # today: a=1, v=da/dt = H0

    # Terminal events
    def _singularity(t, y): return y[0] - AMIN
    _singularity.terminal  = True
    _singularity.direction = -1

    def _runaway(t, y): return y[0] - AMAX
    _runaway.terminal  = True
    _runaway.direction = 1

    # ── backward (Big Bang) ──
    sb = solve_ivp(rhs, [0, -tmax], y0, events=[_singularity],
                   dense_output=True, max_step=0.5, rtol=1e-8, atol=1e-10)
    nb_ = max(int(abs(sb.t[-1]) / 0.02), 50)
    tb = np.linspace(sb.t[-1], 0, nb_)
    ab = np.maximum(sb.sol(tb)[0], 0.0)

    # ── forward (future) ──
    sf = solve_ivp(rhs, [0, tmax], y0, events=[_singularity, _runaway],
                   dense_output=True, max_step=0.5, rtol=1e-8, atol=1e-10)
    nf_ = max(int(abs(sf.t[-1]) / 0.02), 50)
    tf = np.linspace(0, sf.t[-1], nf_)
    af = np.maximum(sf.sol(tf)[0], 0.0)

    # stitch
    t_all = np.concatenate([tb, tf[1:]])
    a_all = np.concatenate([ab, af[1:]])

    age = -tb[0]
    is_crunch = len(sf.t_events[0]) > 0
    if not is_crunch and sf.y.shape[1] > 0:
        v_final = sf.y[1, -1]
        if v_final < 0:
            is_crunch = True
    if is_crunch:
        a_all[-1] = 0.0
        fate = "Big Crunch"
    else:
        q0 = 0.5 * Om + Or - Ol
        fate = "Accelerating expansion" if q0 < 0 else "Decelerating expansion"

    return t_all, a_all, Ok, age, fate


# ═══════════════════════════════════════════════════════════
# Fixed random positions for galaxies and background stars
# ═══════════════════════════════════════════════════════════
_rng = np.random.default_rng(42)
_NG = 25
_gr = _rng.uniform(0.12, 0.88, _NG)
_gt = _rng.uniform(0, 2*np.pi, _NG)
_gx, _gy = _gr * np.cos(_gt), _gr * np.sin(_gt)

_NS = 100
_sx  = _rng.uniform(-2.4, 2.4, _NS)
_sy  = _rng.uniform(-2.4, 2.4, _NS)
_ssz = _rng.uniform(0.2, 1.2, _NS)
_sal = _rng.uniform(0.2, 0.6, _NS)


# ═══════════════════════════════════════════════════════════
# Visualization
# ═══════════════════════════════════════════════════════════
def _draw(t_all, a_all, Ok, age, fate, Or, Om, Ol, H0, t_now):
    """Draw a(t) curve (left) + interactive globe (right)."""
    a_now = float(np.interp(t_now, t_all, a_all))

    # current phase
    idx = min(np.searchsorted(t_all, t_now), len(t_all) - 1)
    expanding = a_all[idx] >= a_all[max(idx - 1, 0)]
    ipk = int(np.argmax(a_all))
    has_crunch = (fate == "Big Crunch")

    fig = plt.figure(figsize=(14, 6.5))

    # ─── LEFT: a(t) plot ───────────────────────────────
    ax = fig.add_subplot(1, 2, 1)

    if has_crunch:
        ax.plot(t_all[:ipk+1], a_all[:ipk+1], lw=2.5, c='#2196F3',
                label='Expanding')
        ax.plot(t_all[ipk:], a_all[ipk:], lw=2.5, c='#F44336',
                ls='--', label='Collapsing')
        ax.plot(t_all[ipk], a_all[ipk], 'o', c='#FF9800', ms=11,
                mec='darkred', mew=2, zorder=5, label='Turnaround')
        ax.fill_between(t_all[ipk:], 0, a_all[ipk:],
                        alpha=0.07, color='red')
    else:
        m = t_all <= 0
        ax.plot(t_all[m],  a_all[m],  lw=2.5, c='#2196F3', label='Past')
        ax.plot(t_all[~m], a_all[~m], lw=2.5, c='#9C27B0', label='Future')

    # time cursor
    ax.axvline(t_now, c='#FF9800', lw=2, alpha=0.6, ls='--')
    ax.plot(t_now, a_now, 'o', c='#FF9800', ms=10, mec='k', mew=1.5,
            zorder=6, label=f't = {t_now:.1f} Gyr')

    ax.axvline(0, c='grey', lw=0.8, alpha=0.3, ls=':')
    ax.axhline(1, c='grey', lw=0.8, alpha=0.3, ls=':')

    ax.set_xlabel('Cosmic time  t [Gyr]   (t = 0 is today)', fontsize=11)
    ax.set_ylabel('Scale factor  a(t)   (a = 1 today)',       fontsize=11)
    ax.set_title('Scale Factor Evolution', fontsize=13, weight='bold')
    ax.set_ylim(0, max(a_all.max() * 1.1, 1.3))
    ax.legend(loc='upper left', fontsize=8.5, framealpha=0.9)
    ax.grid(True, alpha=0.2, ls=':')

    info = (f"\u03a9r = {Or:.4f}   \u03a9m = {Om:.3f}\n"
            f"\u03a9\u039b = {Ol:+.3f}  \u03a9k = {Ok:+.3f}\n"
            f"H\u2080 = {H0:.1f} km s\u207b\u00b9 Mpc\u207b\u00b9\n"
            f"Age \u2248 {age:.2f} Gyr\n"
            f"Fate: {fate}")
    ax.text(0.98, 0.98, info, transform=ax.transAxes, va='top', ha='right',
            fontsize=8.5, family='monospace',
            bbox=dict(boxstyle='round,pad=0.5', fc='#E3F2FD',
                      ec='#1565C0', lw=1.5, alpha=0.9))

    # ─── RIGHT: Universe globe ─────────────────────────
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_xlim(-2.5, 2.5)
    ax2.set_ylim(-2.8, 2.8)
    ax2.set_aspect('equal')
    ax2.set_facecolor('#0d1117')
    ax2.set_xticks([]); ax2.set_yticks([])
    for sp in ax2.spines.values():
        sp.set_visible(False)

    # background stars
    for bx, by, bsz, bal in zip(_sx, _sy, _ssz, _sal):
        ax2.plot(bx, by, '.', c='white', ms=bsz, alpha=bal * 0.5)

    # globe radius: linear in a(t), capped for nice display
    RMAX  = 1.8
    ACLIP = max(min(a_all.max(), 5.0), 1.5)
    R = max(RMAX * min(a_now, ACLIP) / ACLIP, 0.04)

    col = '#F44336' if (has_crunch and not expanding) else '#42A5F5'
    phase = 'COLLAPSING' if (has_crunch and not expanding) else 'EXPANDING'

    # glow layers
    for fac, al in [(1.20, 0.05), (1.10, 0.10)]:
        ax2.add_patch(plt.Circle((0, 0), R * fac, fc=col, ec='none', alpha=al))

    # sphere body + outline
    ax2.add_patch(plt.Circle((0, 0), R, fc=col, ec='white', lw=1.5, alpha=0.3))
    phi = np.linspace(0, 2*np.pi, 200)
    ax2.plot(R*np.cos(phi), R*np.sin(phi), c='white', lw=1.5, alpha=0.5)

    # latitude lines (orthographic projection)
    for deg in [-45, 0, 45]:
        yL = R * np.sin(np.radians(deg))
        hw = R * np.cos(np.radians(deg))
        xs = np.linspace(-hw, hw, 80)
        ax2.plot(xs, np.full_like(xs, yL), c='white', lw=0.5, alpha=0.15)

    # longitude lines
    th = np.linspace(-np.pi/2, np.pi/2, 80)
    for deg in [-45, 0, 45]:
        ax2.plot(R*np.cos(th)*np.sin(np.radians(deg)),
                 R*np.sin(th), c='white', lw=0.5, alpha=0.15)

    # 3-D specular highlight
    ax2.add_patch(plt.Circle((-0.3*R, 0.3*R), 0.25*R,
                              fc='white', ec='none', alpha=0.1))

    # galaxies (comoving coords scaled by globe radius)
    for gx_, gy_ in zip(_gx, _gy):
        px, py = gx_ * R, gy_ * R
        if px**2 + py**2 < (0.95 * R)**2:
            ax2.plot(px, py, '*', c='#FFD54F',
                     ms=max(3, 7*R/RMAX), alpha=0.9,
                     mec='#FFA000', mew=0.3)

    # singularity labels + burst arrows
    if a_now < 0.03:
        lbl = 'BIG CRUNCH' if t_now > 0 else 'BIG BANG'
        bcol = '#EF5350' if t_now > 0 else '#FF8A65'
        ax2.text(0, 0, lbl, ha='center', va='center', fontsize=15,
                 weight='bold', color=bcol,
                 bbox=dict(boxstyle='round,pad=0.5', fc='#1a1a2e',
                           ec=bcol, lw=2, alpha=0.95))
        for ang in np.linspace(0, 2*np.pi, 8, endpoint=False):
            dx, dy = 0.6*np.cos(ang), 0.6*np.sin(ang)
            if t_now > 0:
                ax2.annotate('', xy=(0, 0), xytext=(dx, dy),
                             arrowprops=dict(arrowstyle='->',
                                             color=bcol, lw=1.5))
            else:
                ax2.annotate('', xy=(dx, dy), xytext=(0, 0),
                             arrowprops=dict(arrowstyle='->',
                                             color=bcol, lw=1.5))
    else:
        c2 = '#EF5350' if (has_crunch and not expanding) else '#64B5F6'
        ax2.text(0, R + 0.5, phase, ha='center', fontsize=11,
                 weight='bold', color=c2,
                 bbox=dict(boxstyle='round,pad=0.3', fc='#1a1a2e',
                           ec=c2, lw=1.5, alpha=0.9))

    ax2.text(0, -2.0, f'a = {a_now:.3f}', ha='center',
             fontsize=12, weight='bold', color='white')
    ax2.text(0, -2.35, f't = {t_now:+.1f} Gyr from today',
             ha='center', fontsize=9, color='#aaa')
    ax2.set_title('Universe', fontsize=13, weight='bold',
                  color='white', pad=12)

    plt.tight_layout()
    plt.show()


# ═══════════════════════════════════════════════════════════
# Interactive widgets
# ═══════════════════════════════════════════════════════════
_S = {}                          # cached integration result

sty  = {'description_width': '50px'}
slay = widgets.Layout(width='300px')

w_Om = widgets.FloatSlider(
    value=0.31, min=0, max=3, step=0.01,
    description='\u03a9m', readout_format='.3f',
    style=sty, layout=slay, continuous_update=False)
w_Or = widgets.FloatSlider(
    value=9e-5, min=0, max=0.01, step=1e-4,
    description='\u03a9r', readout_format='.4f',
    style=sty, layout=slay, continuous_update=False)
w_Ol = widgets.FloatSlider(
    value=0.69, min=-2, max=3, step=0.01,
    description='\u03a9\u039b', readout_format='.3f',
    style=sty, layout=slay, continuous_update=False)
w_H0 = widgets.FloatSlider(
    value=70, min=30, max=100, step=0.5,
    description='H\u2080', readout_format='.1f',
    style=sty, layout=slay, continuous_update=False)
w_t  = widgets.FloatSlider(
    value=0, min=-15, max=100, step=0.1,
    description='t [Gyr]', readout_format='.1f',
    style={'description_width': '55px'},
    layout=widgets.Layout(width='660px'),
    continuous_update=True)

ui = widgets.VBox([
    widgets.HTML('<b style="font-size:14px">Friedmann Parameters</b>'),
    widgets.HBox([w_Om, w_Or]),
    widgets.HBox([w_Ol, w_H0]),
    widgets.HTML('<b style="font-size:14px">Time Explorer</b>&ensp;'
                 '<span style="color:grey;font-size:12px">'
                 'drag to scrub through cosmic history</span>'),
    w_t,
])
out = widgets.Output()


def _recompute():
    t, a, Ok, age, fate = integrate(
        w_Or.value, w_Om.value, w_Ol.value, w_H0.value)
    _S.update(t=t, a=a, Ok=Ok, age=age, fate=fate)

def _refresh():
    if not _S:
        return
    with out:
        clear_output(wait=True)
        _draw(_S['t'], _S['a'], _S['Ok'], _S['age'], _S['fate'],
              w_Or.value, w_Om.value, w_Ol.value, w_H0.value,
              np.clip(w_t.value, _S['t'][0], _S['t'][-1]))

def _on_phys(*_a):
    _recompute()
    ta = _S['t']
    w_t.unobserve(_on_time, names='value')
    w_t.min = -999.0
    w_t.max = round(float(ta[-1]) + 1, 1)
    w_t.min = round(float(ta[0])  - 1, 1)
    w_t.value = float(np.clip(w_t.value, w_t.min, w_t.max))
    w_t.observe(_on_time, names='value')
    _refresh()

def _on_time(*_a):
    _refresh()

for _w in (w_Om, w_Or, w_Ol, w_H0):
    _w.observe(_on_phys, names='value')
w_t.observe(_on_time, names='value')

display(ui, out)
_on_phys()


Output()

### Discussion prompts
1. Find a parameter choice that **recollapses**. What sign of $\Omega_\Lambda$ makes this easy?
2. Can you make a universe with $\Omega_k>0$ (open geometry) that still accelerates forever?
3. Why does the "age" depend on $H_0$ as well as the $\Omega$'s?
4. Increase $\Omega_r$ — at what epochs does radiation dominate? Why is its effect on the *future* fate negligible?
